<a href="https://colab.research.google.com/github/michaelhyatt/parsing-data-with-jupyter/blob/master/Parsing_text_data_with_index_pipelines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting all the benefits of schema-on-write with ingest pipelines built iteratively.

## tl;dr
This is a tutorial walkthrough that starts with the data ingested into Elasticsearch as text and walks you though the steps to iteratively build the ingest pipelines needed to convert the text into structured data suitable for analysis and queries in Elastic stack.

## Intro

I've seen few customers recently [migrating to Elastic from Splunk](https://www.elastic.co/splunk-alternative) and observed them coming to terms with the new paradigm of paying more attention to the ingested data. In their previous life with the tools that work on the schema-on-read principle, all they had to do was to send the data unparsed and only worry about parsing it when the time came to search and alert on it. Obviously, this approach resulted in slow search queries and increased demand on the infrastructure e.g. CPU utilisation, but their life was easy when ingesting data. Eventually, the dissatisfaction with poor query performance (among other things) pushed the customers to adopt Elastic greatly improving the search experience, but this improvement was accompanied by having to pay closer attention to how the data is being ingested. Since there is no miracles in engineering, only trade-offs, Elastic customers are adapting to paying the "ingest tax" once, at ingest time instead of paying it every time they are searching through their data. In my opinion, it is a trade-off that is well worth it. I mean, if you are searching, threat hunting and looking for a suspected attacker dwelling in your network all while being super angry at the queries taking their time for minutes and hours, you won't find a consolation in the amount of effort you saved by being lazy and not paying attention to your data at ingest. Still, I felt more could be done to describe the process of how it is possible to build the ingest iteratively starting from ingesting the data as text first, exploring it in Elastic using text search and iteratively defining the ingest pipelines that can be later used to set up properly parsed data ingest that can be further used for visualisations, dashboards, machine learning and alerting.

This blog post extends the ideas formulated in [another Elatic blog post](https://www.elastic.co/blog/schema-on-write-vs-schema-on-read) that outlines different levels of schema-on-write compliance, and describes the process of moving from "text-only" ingest to a more structured data ingest with more comprehensive data parsing at ingest time using [ingest pipelines](https://www.elastic.co/guide/en/elasticsearch/reference/current/pipeline.html). So, this blog post details the step-by-step process of starting with the data loosely ingested as text and results in fully parsed index, and [ingest pipeline](https://www.elastic.co/guide/en/elasticsearch/reference/current/pipeline.html) that was built and tested iteratively. Going forward, the ingest layer can be configured to use this [ingest pipeline](https://www.elastic.co/guide/en/elasticsearch/reference/current/pipeline.html) to produce a fully structured index. So, one can start with a very lax approach of just sending the text into Elastic cluster and refining it later to get all the benefits of schema-on-write: super-fast search, visualisations and alerting. So, get the benefits of not worrying too much about the ingest initially, but eventually getting all the benefits iteratively later.

This blog post was written as [Jupyter](https://jupyter.org/) notebook because it is also possible to get the code and run it as is. I was using [Google Colab](https://colab.research.google.com/notebooks/intro.ipynb) because it was super easy to get started with it in a fully hosted environment in the cloud, which is also free. For on-prem deployments where Google Colab is not an option, [Jupyter](https://jupyter.org/) can be set up locally. Using [Elastic Cloud](https://cloud.elastic.co/) as another "Easy" button that I like to use and will be using here, but with very few modifications the code in this post can run on any Elastic cluster. In any case, [Elastic Cloud trial](https://www.elastic.co/cloud/elasticsearch-service/signup) is available for free for the first 14 days.

On the techical side, the code makes use of 2 Elasticsearch Python clients, [elasticsearch-py](https://elasticsearch-py.readthedocs.io/en/master/) and [elasticsearch-dsl](https://elasticsearch-dsl.readthedocs.io/en/latest/). The former is a low level python client that covers the entirety of Elasticsearch APIs. The latter is a more concise way of expressing things like index mapping and creation, as well as query APIs, but it doesn't cover (yet) the entirety of what [Elasticsearch APIs](https://www.elastic.co/guide/en/elasticsearch/reference/current/rest-apis.html) have to offer. So, it is [elasticsearch-dsl](https://elasticsearch-dsl.readthedocs.io/en/latest/) where possible falling back on [elasticsearch-py](https://elasticsearch-py.readthedocs.io/en/master/) for completeness. I am also using [pandas](https://pandas.pydata.org/pandas-docs/stable/index.html) to visualise the outputs because looking at large nested JSON messages hurts my eyes. Also, [pandas](https://pandas.pydata.org/pandas-docs/stable/index.html) is a great way to slice and dice the data, but I am not making much use of it here. I am also making use of [Python list and dictionary comprehensions](https://www.digitalocean.com/community/tutorials/understanding-list-comprehensions-in-python-3dec), because these square bracket expressions are so neat to transform collections.

All the code (including [this blog post](https://github.com/michaelhyatt/parsing-data-with-jupyter/blob/master/Parsing_text_data_with_index_pipelines.ipynb) written as [Jupyter](https://jupyter.org/) notebook) and sample data used in this post is available in this [GitHub repository](https://github.com/michaelhyatt/parsing-data-with-jupyter).


## Loading the sample data

For this post I used a sample multiline log file (available [here](https://github.com/michaelhyatt/parsing-data-with-jupyter/blob/master/ossec-alerts.log)). I wanted to have a text that is somewhat complex and spans across multiple lines. This is an example of a single entry from the file. It starts with `** ` statement and ends with the next `** ` token. Feel free to try the tips from this blog on your own sample.
```
...
** Alert 1585118850.94922: - syslog,sshd,invalid_login,authentication_failed,pci_dss_10.2.4,pci_dss_10.2.5,pci_dss_10.6.1,gpg13_7.1,gdpr_IV_35.7.d,gdpr_IV_32.2,hipaa_164.312.b,nist_800_53_AU.14,nist_800_53_AC.7,nist_800_53_AU.6,
2020 Mar 25 06:47:30 haran-ids-lab->/var/log/secure
Rule: 5710 (level 5) -> 'sshd: Attempt to login using a non-existent user'
Src IP: 106.12.156.236
Src Port: 60156
Mar 25 06:47:29 haran-ids-lab sshd[26252]: Invalid user sinus from 106.12.156.236 port 60156
...

```
I loaded the data using [filebeat](https://www.elastic.co/guide/en/beats/filebeat/current/index.html). Apart from only configuring the input to be taken from `stdin` and connection details to Elastic cluster, I only had to configure 2 other things:
* Handling of [multiline messages](https://www.elastic.co/guide/en/beats/filebeat/current/multiline-examples.html).
* Filtering our all the fields, apart from `message` to simulate a text only initial index.

These are the fragments of configuration to be added to `filebeat.yml`:

```yaml
# Multiline support on data received through stdin
filebeat.inputs:
  - type: stdin
    multiline.pattern: '^\*\* '
    multiline.negate: true
    multiline.match: after
...
# Just send the message field
processors:
 - include_fields:
    fields: ["message"]
```

So, to ingest the data using filebeat assuming `filebeat.yml` is in place, just run the following command:
```
$ cat ossec-alerts.log | ./filebeat -e
```
I find it quite useful to use the `console` output to get the format right instead of sending data directly into Elasticsearch, but in case you want to reload the data, just delete the `filebeat-*` index that filebeat creates. Go to Dev tools part of Kibana and run `DELETE filebeat-*` API command. Obviously, be careful not to delete valuable indexes in production.

## Elasticsearch Python clients: low-level and DSL

I use of 2 Elasticsearch Python clients, [elasticsearch-py](https://elasticsearch-py.readthedocs.io/en/master/) and [elasticsearch-dsl](https://elasticsearch-dsl.readthedocs.io/en/latest/). The former is a low level python client that covers the entirety of Elasticsearch APIs. The latter is a more concise way of expressing things like index mapping and creation, as well as query APIs, but it doesn't cover (yet) the entirety of what [Elasticsearch APIs](https://www.elastic.co/guide/en/elasticsearch/reference/current/rest-apis.html) have to offer. So, it is [elasticsearch-dsl](https://elasticsearch-dsl.readthedocs.io/en/latest/) where possible falling back on [elasticsearch-py](https://elasticsearch-py.readthedocs.io/en/master/) for completeness.

First, let's make sure all the required dependencies are installed:

In [0]:
!pip install elasticsearch-dsl elasticsearch pandas

Next, we will run a search query to bring in the data from `filebeat-*` index that will be used to test our ingest pipeline. We are going to get the records from the source index as is to be passed to the [simulate API](https://www.elastic.co/guide/en/elasticsearch/reference/master/simulate-pipeline-api.html#simulate-pipeline-api-request-body-ex) in Elasticsearch later on.

The code below uses Elastic Cloud and connects to the cluster using `cloud.id` and `cloud.auth`. It is easy to modify the code to use the direct Elasticsearch REST API.

In [0]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search

# Create a connection
es = Elasticsearch(
    cloud_id="test1:YXVzdHJhbGlhLXNvdXRoZWFzdDEuZ2NwLmVsYXN0aWMtY2xvdWQuY29tJDliYTI3ZDIxMmZkZTQ0MTlhNzg5NWVmNDU3YWI3ZDYxJDk3ZmEzODllMTAyYjRlZGNhMTE2MTUyMTA0ODEyMzZk",
    http_auth=('filebeat_reader', 'read_that_beat'))

# Define search using dsl API
s = Search().using(es).index("filebeat-*")

# How many resuts to return
s = s.extra(size=5)

# What type of query to run
s = s.query('match_all')

response = s.execute()

# Let's just check that the request didn't time out and returned successfully.
# False is a good response BTW. Means that request didn't time out.
response.timed_out

In [0]:
# Create the docs array of dicts to be passed into the simulate API as JSON
docs = [{'_source': doc['_source']} for doc in response.to_dict()['hits']['hits']]

# Display the resulting structure
docs

## Parsing using ingest pipelines

In this tutorial, I am using [ingest pipelines](https://www.elastic.co/guide/en/elasticsearch/reference/master/pipeline.html) to parse the data. Ingest pipelines, for those unaware of them, are pipelines consisting of processors that can manipulate the data and change it as it flows from input to output. Ingest pipelines are executed on cluster nodes that have [ingest role](https://www.elastic.co/guide/en/elasticsearch/reference/master/ingest.html), or a setting `node.ingest: true` in their `elasticsearch.yml` config file. The ingest nodes can either be dedicated or combined with other roles, depends on how busy your cluster is. The main benefits of using the ingest pipelines are:
* No need for additional software to be used, such as beats or Logstash.
* Ingest pipelines run in the cluster itself.
* Elasticsearch APIs allow using ingest pipelines in data transofrmations, such as [update_by_query](https://www.elastic.co/guide/en/elasticsearch/reference/current/docs-update-by-query.html) or [reindex](https://www.elastic.co/guide/en/elasticsearch/reference/current/docs-reindex.html) APIs.

Other ways of achieving the same outcomes while staying within the Elastic stack will be using [Logstash](https://www.elastic.co/logstash) or [processors within beats](https://www.elastic.co/guide/en/beats/filebeat/current/defining-processors.html).

In this tutorial, I will build the pipelines iteratively, step by step, validating the functionality of the pipelines using the [pipeline simulation API](https://www.elastic.co/guide/en/elasticsearch/reference/master/simulate-pipeline-api.html). This API allows us to take a pipeline definition and a collection of sample records and get the results of the pipeline execution on the collection of records. This is a good way to iteratively test the pipeline definition before making it permanent in Elasticsearch. The data for the simulation will come from our source index, so when we are happy with the results that our pipeline produces, we will be able to make the change permanent by creating a pipeline from the definition we tested.

Let's have a look how pipeline definition and simulate API will work in our example. Let's create a pipeline definition that only has one processor: adding a field to each processed event that will be named `test` and will have a value `value123`:

In [0]:
pipeline = {
  "processors": [
    {
      "set": {
          "field": "test",
          "value": "value123"
      }
    }
  ]
}

Remember the `docs` variable we defined in previous steps? Thsi variable contains a collection of documents that we will pass to the simulate APIs. These documents came straight from our source index `filebeat-*` and they contain `@timestamp` and `message` fields only. `message` field is the one we are going to turn from text to a collection of parsed fields.

In [0]:
docs

Now, let's invoke the [simulate API](https://www.elastic.co/guide/en/elasticsearch/reference/master/simulate-pipeline-api.html) and check out the results we are getting.

In [0]:
simulate_request = {
    "pipeline": pipeline,
    "docs": docs
}

# Invoke the Simulate API
response = es.ingest.simulate(body=simulate_request)

# Display the resulting documents
[doc['doc']['_source'] for doc in response['docs']]

We can kind of see the new field `test: value123` added to each document, but it is not a great experience trying to squint at JSON text. This is where we are going to use `pandas`. 

## Using pandas in Jupyter

Let's take the response JSON and visualise it in `pandas`. Note that I am only visualising the fields other than `message` and `@timestamp`. It allows me to see only the newly created fields. Now, the new field `test` shows neatly as a table.

In [0]:
import pandas as pd

# Make sure pandas tables don't truncate the fields
pd.options.display.max_colwidth = None

doc_list = [doc['doc']['_source'] for doc in response['docs']]

df = pd.DataFrame(doc_list)

df[ [x for x in df.columns if x not in ["message", "@timestamp"]] ]

Let's define some helper functions to call the simulate API returning the dataframe and to display the data in columns.

In [0]:
def test_pipeline(in_pipeline, in_docs):
  '''
    Run the simulate API with the provided pipeline and docs collection.
    Returns pandas DataFrame with results.
  '''

  simulate_request = {
    "pipeline": in_pipeline,
    "docs": in_docs
  }

  response = es.ingest.simulate(body=simulate_request)

  doc_list = [doc['doc']['_source'] for doc in response['docs']]

  return pd.DataFrame(doc_list)
  

def print_new_fields(in_df, print_timestamp = False):
  '''
    Filters out message and @timestamp columns
  '''
  return in_df[[x for x in in_df.columns if x != "message" and (x != "@timestamp" or print_timestamp)]]

Let's start building our pipeline. Start with the [dissect processor](https://www.elastic.co/guide/en/elasticsearch/reference/master/dissect-processor.html). Note how we create all the parsed fields and there is `rest` field that contains the remainder of the `message` field that it is still unparsed.

In [0]:
# Let's have a sample evant we need to parse in front of us:
# "** Alert 1585107670.35304: - syslog,sshd,invalid_login,authentication_failed,pci_dss_10.2.4,pci_dss_10.2.5,pci_dss_10.6.1,gpg13_7.1,gdpr_IV_35.7.d,gdpr_IV_32.2,hipaa_164.312.b,nist_800_53_AU.14,nist_800_53_AC.7,nist_800_53_AU.6,\n2020 Mar 25 03:41:10 haran-ids-lab->/var/log/secure\nRule: 5710 (level 5) -> 'sshd: Attempt to login using a non-existent user'\nSrc IP: 124.251.110.148\nSrc Port: 58112\nMar 25 03:41:09 haran-ids-lab sshd[7190]: Invalid user bg from 124.251.110.148 port 58112\n"
pipeline = {
  "processors": [
    {
      "dissect": {
          "field": "message",
          "pattern": "** Alert %{time_sec}.%{alert_id}: - %{rest}"
      }
    }
  ]
}

# Calling our helper function #1
df = test_pipeline(pipeline, docs)

# Calling our helper function #2
print_new_fields(df)

Our flow is working. Let's enhance the dissect `pattern` expression and retrieve all the fields we are interested in. This is where we want to spend time iterating to get our pipeline to handle all the cases that our data may present to us.

In [0]:
# Let's have a sample evant we need to parse in front of us:
# "** Alert 1585107670.35304: - syslog,sshd,invalid_login,authentication_failed,pci_dss_10.2.4,pci_dss_10.2.5,pci_dss_10.6.1,gpg13_7.1,gdpr_IV_35.7.d,gdpr_IV_32.2,hipaa_164.312.b,nist_800_53_AU.14,nist_800_53_AC.7,nist_800_53_AU.6,\n2020 Mar 25 03:41:10 haran-ids-lab->/var/log/secure\nRule: 5710 (level 5) -> 'sshd: Attempt to login using a non-existent user'\nSrc IP: 124.251.110.148\nSrc Port: 58112\nMar 25 03:41:09 haran-ids-lab sshd[7190]: Invalid user bg from 124.251.110.148 port 58112\n"
pipeline = {
  "processors": [
    {
      "dissect": {
          "field": "message",
          "pattern": "** Alert %{time_sec}.%{alert_id}: - %{tag_csv}\n%{year} %{month} %{day} %{time} %{host}->%{path}\nRule: %{rule_id} (%{level}) -> '%{rule_msg}'\nSrc IP: %{source_ip}\n%{rest}"
      }
    }
  ]
}

df = test_pipeline(pipeline, docs)

print_new_fields(df)

We can see there is a comma-separated values in one of the fields. 
There are also some records that start with `Src Port:` that contain port number that can be retrieved, but it only applies to certain fields, so we will add a conditional `if` statement to our second `dissect` processor. This is also a part of the iterative process where we are adding more processors to the ingest pipeline. Check out the conditional statement in the `if` element of the second `dissect` processor. It uses [`Painless` scripting language](https://www.elastic.co/guide/en/elasticsearch/reference/master/modules-scripting-painless.html).

In [0]:
pipeline = {
  "processors": [
    {
      "dissect": {
          "field": "message",
          "ignore_failure": True,
          "pattern": "** Alert %{time_sec}.%{alert_id}: - %{tag_csv}\n%{year} %{month} %{day} %{time} %{host}->%{path}\nRule: %{rule_id} (%{level}) -> '%{rule_msg}'\nSrc IP: %{source_ip}\n%{rest}"
      }
    },
    {
      "dissect": {
          "if": "ctx.rest?.substring(0, 8) == 'Src Port'",
          "ignore_failure": True,
          "field": "rest",
          "pattern": "Src Port: %{source_port}\n%{rest}\n"
      }
    },
    {
     "split": {
         "field": "tag_csv",
         "separator": ","
     }
    }
  ]
}

df = test_pipeline(pipeline, docs)

print_new_fields(df)

Let's finish our pipeline by parsing the timestamp and only leaving the fields we are interested in. But first, let's see what fields we have:

In [0]:
df.columns.to_list()

We will filter out the fields we are not interested in, add a `date` processor parsing the timestamp from the data and add `on_failure` flow to handle parsing exceptions, or in our case, we will simply drop the offending events. Not that you should do that with real data. You probably want to handle it more gracefully. For example, tag it with a `ingestparsingfailure`, so you can later check how many log entries didn't fit through your ingest pipeline. This is another feature that can allow you to have a dashboard showing you the health of the ingest, so you know the reason behind your alerts staying quiet: is it because everything is normal, or because the data is simply not there to raise the alerts. No need to run synthetic queries to check for that data to be present. 

In [0]:
pipeline = {
  "description": "This is a final version of the processing pipeline",
  "processors": [
    {
      "dissect": {
        "field": "message",
        "pattern": "** Alert %{time_sec}.%{alert_id}: - %{tag_csv}\n%{year} %{month} %{day} %{time} %{host}->%{path}\nRule: %{rule_id} (%{level}) -> '%{rule_msg}'\nSrc IP: %{source_ip}\n%{rest}"
      }
    },
    {
      "dissect": {
        "if": "ctx.rest?.substring(0, 8) == 'Src Port'",
        "field": "rest",
        "pattern": "Src Port: %{source_port}\n%{rest}\n"
      }
    },
    {
      "split": {
        "field": "tag_csv",
        "separator": ","
      }
    },
    {
      "date": {
        "field": "time_sec",
        "formats": ["UNIX"]  
      }
    },
    {
      "remove": {
        "field": ['rest', 'year', 'level', 'message', 'time_sec', 'month', 'time', 'day']
      }
    }
  ],
  "on_failure": [
    {
      "drop": {}
    }
  ]
}

df = test_pipeline(pipeline, docs)

print_new_fields(df, print_timestamp=True)

## Once happy, re-index into another index or use update_by_query API

Now that our pipeline is producing satisfactory simulated results, let's create it permanently in the cluster. We will then use it to reindex our `filebeat-*` index that contains only text field `message` into another index that will contain the parsed data fields that will have defined types. There are few more APIs that take ingest pipelines, such as [update_by_query](https://www.elastic.co/guide/en/elasticsearch/reference/current/docs-update-by-query.html) that allows modification of the source index in-place by adding the new fields to the existing documents.

Now, let's create our pipeline:

In [0]:
es.ingest.put_pipeline(id="dest1-pipeline", body=pipeline)

Let's create the new index with the new field mappings. These are the columns we will have in the resulting index:

In [0]:
df.columns.to_list()

Now we know the columns that our index needs to have, let's define the index and the mappings:

In [0]:
from elasticsearch_dsl import Keyword, Mapping, Text, Date, Ip, Index

# Delete dest1 index in case it exists when you are re-running this bit.
es.indices.delete("dest1", ignore_unavailable=True)

# Define the mapping
m = Mapping()

m.field("tag_csv", Keyword())
m.field("rule_id", Keyword())
m.field("path", Keyword())
m.field("@timestamp", Date())
m.field("alert_id", Keyword())
m.field("host", Keyword())
m.field("rule_msg", Text())
m.field("source_ip", Ip())
m.field("source_port", Keyword())

# Create the destination index
i = Index("dest1")
i.mapping(m)
i.create(using=es)

# Run reindex using the pipeline we created earlier
request = {
  "source": {
      "index": "filebeat-*"
  },
  "dest": {
      "index": "dest1",
      "pipeline": "dest1-pipeline"
  }
}

es.reindex(body=request)

Let's search the new index and have a look at the data produced by reindexing the data from `filebeat-*` index using the pipeline we created:

In [0]:
s = Search().using(es).index("dest1")

s = s.extra(size=20)

s = s.query('match_all')

response = s.execute()

body = [doc['_source'].to_dict() for doc in response.hits.hits]

pd.DataFrame(body)

## Summary and final thoughts

That's all there is to it. We iteratively built our ingest pipeline using Elasticsearch pipeline simulate API and reindexed our source `filebeat-*` index that contained only text into a structured index called `dest1`. Now, the data in `dest1` can be used not only for search, but also for visualisations, alerting, machine learning and everything else Elastic stack has to offer. But wait, there is more. The pipeline `dest1-pipeline` that we built iteratively can be used to parse the data as it is being ingested into Elasticsearch, so you no longer have to live with unparsed blob of text. Use this pipeline in your filebeat or Logstash configuration to make schema-on-write happen auto-magically. You also now have the correct and tested mapping for your indices that can go straight into the index template you will use for the ingest. 

Congratulations, the ingest work is now done, the ingest tax has been paid, so sit back and enjoy super-fast threat-hunting, searches and visualisations.

*PS: I realised that writing a blog post about software that is also a piece of software you can run is very much like what happened in that episode of Seinfeld when Kramer wrote a coffee table book about coffee tables that also turns into a coffee table... Just a random thought...*
